#### Day 19 - A
Check which designs can be made using combinations of different towels.

In [9]:
#Import Libraries and settings
from copy import deepcopy

settings = {
    "day": 19,
    "test_data": 0
}

In [10]:
#Load Input
def load_input(settings):
    #Derrive input file name
    if settings["test_data"]:
        data_subdir = "test"
    else:
        data_subdir = "actual"

    data_fp = f"./../input/{data_subdir}/{settings["day"]}.txt"

    #Open and read the file
    with open(data_fp) as f:
        lines = f.read().split('\n')

    towels = lines[0].split(", ")

    #Convert towels into dictionary grouped by the first character
    tlu = {}
    for towel in towels:
        start = towel[0]
        if start not in tlu.keys():
            tlu[start] = [towel]
        else:
            tlu[start].append(towel)

    designs = lines[2:]

    return towels, tlu, designs

towels, tlu, designs = load_input(settings)

In [11]:
#Attempt to recursively make the design from towels
def match_design(tlu, design):

    #Get the next character in the design
    next_char = design[0]
    remaining_design = len(design)

    #Check there are towels starting with this pattern
    if next_char not in tlu.keys():
        return False

    #Try all possible options
    for towel in tlu[next_char]:
        #If this towel matches the next few characters of the design
        if len(towel) < remaining_design:
            if towel == design[0:len(towel)]:
                res = match_design(tlu, design[len(towel):])
                if res:
                    return towel + res


        #If this towel matches the rest of the design    
        elif len(towel) == remaining_design:
            if towel == design:
                return towel
            
    return False

In [12]:
#Get all towels of a given length
#When exact is false it also returns towels shorter than the specified length
def all_towels_of_len_n(tlu, length, exact=True):
    matches = []

    #Get all towels of the required length
    for key in tlu.keys():
        for towel in tlu[key]:
            if exact:
                if len(towel) == length:
                    matches.append(towel)
            else:
                if len(towel) <= length:
                    matches.append(towel)

    #If exact is False, return towels as a dictionary grouped by length
    if exact == False:
        tlu_p = {}
        for piece in matches:
            start = piece[0]
            if start not in tlu_p.keys():
                tlu_p[start] = [piece]
            else:
                tlu_p[start].append(piece)
        return tlu_p
    else:
        return matches

#Convert a full towel lookup
def rebuild(tlu, max=8, status=True):

    if status:
        print("Rebuilding towel lookup dictionary...")

    #Create a copy to avoid overwriting the original
    tl2 = deepcopy(tlu)

    #For each possible towel length (in ascending order)
    for length in range(2, max+1):
        redundant_towels = []
        #Get all towels of current length
        towel_targets = all_towels_of_len_n(tl2, length, exact=True)
        #Get all towels of a shorter length
        towel_pieces = all_towels_of_len_n(tl2, length-1, exact=False)

        #For each towel or the specified length
        for target in towel_targets:
            #Attempt to make the towel using smaller ones
            if match_design(towel_pieces, target):
                #If the towel can be made from smaller towels, it is redundant
                redundant_towels.append(target)

        #Remove redundant towels from tlu
        for key in tl2.keys():
            tl2[key] = list(filter(lambda x: x not in redundant_towels, tl2[key]))
        
        #Print processing status
        if status:
            print("Towels of length", length, "processed.")

    if status:
        print()
    return tl2

In [15]:
def count_possible_designs(towel_lookup, designs, status=True):
    count = 0

    if status:
        print("Processing", len(designs), "designs:")

    #For each design, check if it can be made using the towel lookup dict
    for idx, design in enumerate(designs):
        if status and idx%100 == 0:
            print("Processing design", str(idx)+"...")

        res = match_design(towel_lookup, design)
        if res:
            count += 1

    return count

In [16]:
tl2 = deepcopy(tlu)
tl2 = rebuild(tl2)
print(count_possible_designs(tl2, designs))

Rebuilding towel lookup dictionary...
Towels of length 2 processed.
Towels of length 3 processed.
Towels of length 4 processed.
Towels of length 5 processed.
Towels of length 6 processed.
Towels of length 7 processed.
Towels of length 8 processed.

Processing 400 designs:
Processing design 0...
Processing design 100...
Processing design 200...
Processing design 300...
311


In [17]:
for key in tl2.keys():
    tl2[key] = sorted(tl2[key], key=len)
    print(key, tl2[key])

r ['rg', 'rr', 'rb', 'rw', 'ru', 'rbr', 'rgr', 'rrr', 'rwr', 'rur', 'rbur', 'rgur', 'rwbwwur']
g ['g', 'gr', 'gur', 'gwwur']
w ['w', 'wr']
b ['b', 'br', 'bur', 'bwur', 'bbwgwur']
u ['u', 'uur', 'uwur', 'ugwur']
